In [1]:
# -------------- N-Queens problem ----------------------

import random

# Define the size of the board (number of queens)
n = 8  # You can change this to any N-Queens size
population_size = 10  # Number of individuals in a population
mutation_rate = 0.1  # Probability of mutation

def calculate_n_queens_fitness(individual):
    """Calculate fitness by counting non-attacking pairs of queens."""
    non_attacking_pairs = 0
    total_pairs = n * (n - 1) // 2  # Maximum possible non-attacking pairs
    # Check for conflicts
    for i in range(n):
        for j in range(i + 1, n):
            if individual[i] != individual[j] and abs(individual[i] - individual[j]) != abs(i - j):
                non_attacking_pairs += 1
    
    return non_attacking_pairs / total_pairs  # Normalize fitness score

def create_random_individual():
    """Generate a random individual (chromosome) representing column positions."""
    return random.sample(range(n), n)  # Unique column positions

def select_parents(population, fitness_scores):
    """Select top 50% of the population based on fitness scores."""
    sorted_population = [board for _, board in sorted(zip(fitness_scores, population), reverse=True)]
    return sorted_population[:len(population) // 2]

def crossover(parent1, parent2):
    """Perform single-point crossover while maintaining unique column positions."""
    point = random.randint(1, n - 2)  # Choose crossover point
    child = parent1[:point] + parent2[point:]
    
    # Ensure unique column positions
    missing = set(range(n)) - set(child)
    duplicates = [col for col in child if child.count(col) > 1]
    
    for i in range(len(child)):
        if child.count(child[i]) > 1:
            child[i] = missing.pop()
    
    return child

def mutate(individual):
    """Randomly swap two positions in the individual to introduce variation."""
    idx1, idx2 = random.sample(range(n), 2)
    individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

def genetic_algorithm():
    """Main Genetic Algorithm function to evolve a solution for the N-Queens problem."""
    population = [create_random_individual() for _ in range(population_size)]
    generation = 0
    best_fitness = 0
    
    while best_fitness < 1.0 and generation < 100:
        # Evaluate fitness
        fitness_scores = [calculate_n_queens_fitness(ind) for ind in population]
        best_fitness = max(fitness_scores)
        print(f"Generation {generation} Best Fitness: {best_fitness}")
        
        # Check for optimal solution
        if best_fitness == 1.0:
            break
        
        # Selection
        parents = select_parents(population, fitness_scores)
        
        # Crossover to create new population
        new_population = [crossover(random.choice(parents), random.choice(parents)) for _ in range(population_size)]
        
        # Mutation
        for i in range(len(new_population)):
            if random.random() < mutation_rate:
                new_population[i] = mutate(new_population[i])
        
        population = new_population
        generation += 1
    
    # Return the best solution
    best_individual = max(population, key=calculate_n_queens_fitness)
    return best_individual, calculate_n_queens_fitness(best_individual)

# Run the Genetic Algorithm
solution, fitness = genetic_algorithm()
print("Best Solution:", solution)
print("Best Fitness:", fitness)



Generation 0 Best Fitness: 0.9642857142857143
Generation 1 Best Fitness: 0.9642857142857143
Generation 2 Best Fitness: 0.9642857142857143
Generation 3 Best Fitness: 0.9285714285714286
Generation 4 Best Fitness: 0.9285714285714286
Generation 5 Best Fitness: 0.9285714285714286
Generation 6 Best Fitness: 0.9285714285714286
Generation 7 Best Fitness: 0.9285714285714286
Generation 8 Best Fitness: 0.9285714285714286
Generation 9 Best Fitness: 0.9642857142857143
Generation 10 Best Fitness: 0.9642857142857143
Generation 11 Best Fitness: 0.9642857142857143
Generation 12 Best Fitness: 0.9642857142857143
Generation 13 Best Fitness: 0.9642857142857143
Generation 14 Best Fitness: 0.9642857142857143
Generation 15 Best Fitness: 0.9642857142857143
Generation 16 Best Fitness: 0.9642857142857143
Generation 17 Best Fitness: 0.9642857142857143
Generation 18 Best Fitness: 0.9642857142857143
Generation 19 Best Fitness: 0.9642857142857143
Generation 20 Best Fitness: 0.9642857142857143
Generation 21 Best Fitn

In [ ]:
#Duty Scheduling Problem Using Genetic Algorithm

import random

# Configuration settings
num_staff = 5  # Number of employees
num_shifts = 21  # Total shifts in a week (7 days * 3 shifts per day)
max_shifts_per_staff = 7  # Maximum shifts a staff member can work in a week
required_staff_per_shift = 2  # Minimum number of staff required per shift
population_size = 10  # Number of schedules in each generation
mutation_rate = 0.1  # Probability of mutation occurring
max_generations = 1000  # Maximum number of iterations

# Fitness function to evaluate schedule quality
def evaluate_fitness(schedule):
    penalty = 0  # Lower penalty means a better schedule
    # Check shift coverage
    for shift in range(num_shifts):
        assigned_count = sum(schedule[staff][shift] for staff in range(num_staff))
        if assigned_count < required_staff_per_shift:
            penalty += (required_staff_per_shift - assigned_count) * 10  # Penalize understaffing
    
    # Check consecutive shifts per staff
    for staff in range(num_staff):
        for shift in range(num_shifts - 1):
            if schedule[staff][shift] == 1 and schedule[staff][shift + 1] == 1:
                penalty += 5  # Penalize consecutive shifts
    
    return penalty

# Function to create a random schedule (chromosome)
def create_random_schedule():
    schedule = [[0] * num_shifts for _ in range(num_staff)]  # Initialize all shifts as unassigned
    for staff in range(num_staff):
        assigned_shifts = random.sample(range(num_shifts), random.randint(3, max_shifts_per_staff))  # Randomly assign shifts
        for shift in assigned_shifts:
            schedule[staff][shift] = 1  # Mark assigned shifts
    return schedule

# Selection function: Selects top 50% of the best schedules
def select_parents(population, fitness_scores):
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population))]  # Sort schedules by fitness
    return sorted_population[:len(population) // 2]  # Return the top half

# Crossover function: Single-point crossover
def crossover(parent1, parent2):
    point = random.randint(0, num_shifts - 1)  # Random crossover point
    child = [parent1[i][:point] + parent2[i][point:] for i in range(num_staff)]  # Combine parents
    return child

# Mutation function: Randomly swap two shifts for a staff member
def mutate(schedule):
    staff = random.randint(0, num_staff - 1)  # Select random staff member
    shift1, shift2 = random.sample(range(num_shifts), 2)  # Select two random shifts
    schedule[staff][shift1], schedule[staff][shift2] = schedule[staff][shift2], schedule[staff][shift1]  # Swap shifts
    return schedule

# Genetic Algorithm Loop
population = [create_random_schedule() for _ in range(population_size)]  # Initialize population

for generation in range(max_generations):
    fitness_scores = [evaluate_fitness(schedule) for schedule in population]  # Calculate fitness for all schedules
    best_fitness = min(fitness_scores)  # Get the best fitness score
    print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")
    
    parents = select_parents(population, fitness_scores)  # Select best parents
    new_population = []
    
    while len(new_population) < population_size:
        parent1, parent2 = random.sample(parents, 2)  # Randomly select two parents
        child = crossover(parent1, parent2)  # Perform crossover
        if random.random() < mutation_rate:
            child = mutate(child)  # Apply mutation
        new_population.append(child)
    
    population = new_population  # Update population

# Find the best schedule
best_schedule = population[fitness_scores.index(min(fitness_scores))]
print("\nBest Schedule (Staff x Shifts):")
for staff in range(num_staff):
    print(f"Staff {staff + 1}: {best_schedule[staff]}")


Generation 1, Best Fitness: 135
Generation 2, Best Fitness: 140
Generation 3, Best Fitness: 130
Generation 4, Best Fitness: 120
Generation 5, Best Fitness: 130
Generation 6, Best Fitness: 125
Generation 7, Best Fitness: 125
Generation 8, Best Fitness: 120
Generation 9, Best Fitness: 120
Generation 10, Best Fitness: 115
Generation 11, Best Fitness: 115
Generation 12, Best Fitness: 115
Generation 13, Best Fitness: 115
Generation 14, Best Fitness: 115
Generation 15, Best Fitness: 115
Generation 16, Best Fitness: 115
Generation 17, Best Fitness: 115
Generation 18, Best Fitness: 115
Generation 19, Best Fitness: 115
Generation 20, Best Fitness: 115
Generation 21, Best Fitness: 115
Generation 22, Best Fitness: 105
Generation 23, Best Fitness: 105
Generation 24, Best Fitness: 105
Generation 25, Best Fitness: 105
Generation 26, Best Fitness: 105
Generation 27, Best Fitness: 105
Generation 28, Best Fitness: 105
Generation 29, Best Fitness: 105
Generation 30, Best Fitness: 105
Generation 31, Best